In [2]:

from library import mathable_solver as s

source_images_dir = "./testare"
output_destination_dir = "./343_Gabroveanu_Razvan"

# Create a solver for each data set and classify the images
# A data set consists of a set of images that are named (data_set_index)_(image_index)
for i in range(0,4):
    solver = s.Solver(i+1,src_images_dir=source_images_dir,output_dest_dir=output_destination_dir)
    solver.solve()

100%|██████████| 50/50 [00:00<00:00, 157.30it/s]
